In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [0]:
df_matched_ext = spark.table("d_dnst_020_silver.silver_03.`032_sellout_customers_ext_matched`")

In [0]:
df_matched_ext.count()

In [0]:
window_spec = Window.partitionBy("unique_id")
window_spec_ordered = Window.partitionBy("unique_id").orderBy(F.col("ext_levenstein_score").desc())

In [0]:
(  
    df_matched_ext
    .filter(F.col("ext_levenstein_score").isNotNull())
    .select("unique_id","customer_name","customer_address","ext_levenstein_score", "ext_match_details", "ext_levenstein_name_address", "ext_BPR_match", "ext_vat_number_match", "ext_place_id_match", "ext_Tax_Number_4_match", "C4S_account_id", "dim_sellout_cust_id")
    .withColumn(
        "source_id_occurence", 
        F.count(F.lit(1)).over(window_spec) # Counts records within the partition
    )
    .withColumn(
        "lev_score_desc", 
        F.row_number().over(window_spec_ordered) # Ranks records by score
    )
    .orderBy(F.desc("source_id_occurence"), F.col("unique_id"), F.asc("lev_score_desc"))
    # .filter(F.col("dim_sellout_cust_id").isin("1022551752"))
    .display()
)

In [0]:
nr_matched = df_matched_ext.filter(F.col("C4S_account_id").isNotNull()).count()
total = df_matched_ext.count()
perc_matched = nr_matched / total
print(f"percentage matched: {perc_matched}")


In [0]:
df_c4s = spark.table("d_dnst_010_bronze.csv_export.c4s_dump")

In [0]:
(
    df_c4s
    .filter(F.col("Account_Id").isin("6042246"))
    .select("Name", "Street","Account_Id", "Postal_Code")
    .display()
)

In [0]:

from pyspark.sql.window import Window

# Define the window partitioned by dim_sellout_cust_id
window_spec = Window.partitionBy("customer_number")

In [0]:
# df_matched_ext_prepped = (
#     df_matched_ext
#     .withColumn("C4S_account_id",
#         F.coalesce(
#             F.col("ext_BPR_match"),
#             F.col("ext_Tax_Number_4_match"),
#             F.col("ext_vat_number_match"),
#             F.col("ext_place_id_match"),
#             F.col("ext_levenstein_name_address")
#         )    
#     )
#     .withColumn("direct_c4S_account_id", F.col("C4S_account_id"))
#     .withColumn("max_indirect_id_in_group", F.first("direct_c4S_account_id", ignorenulls=True).over(window_spec))
# )

In [0]:
(
    df_matched_ext
    .filter(F.col("ext_levenstein_name_address").isNotNull())
    # .filter(~F.col("customer_address").rlike(r"\d"))
    # .filter(F.col("customer_number") == "117598")
    .count()
)

In [0]:
(
    df_c4s
    .filter(F.col("Account_ID") == "5996185")
    .select("Account_ID", "Name", "House_Number", "Street")
    .display()
)

In [0]:
# (
#     df_matched_ext
#     .filter(F.col("ext_levenstein_name_address").isNotNull())
#     .filter(~F.col("customer_address").rlike(r"\d"))
#     .display()
# )

In [0]:
(
    df_matched_ext
    .withColumn("count_dim_sellout_cust_id", F.count("*").over(window_spec))
    .withColumn("count_C4S_account_id", F.count("C4S_account_id").over(window_spec))
    .select("customer_number","ext_levenstein_name_address","unique_id", "dim_sellout_cust_id", "direct_c4S_account_id","C4S_account_id", "customer_name", "customer_address","customer_postal_code", "count_dim_sellout_cust_id","count_C4S_account_id", 
    # "ext_BPR_match", "ext_Tax_Number_4_match", "ext_vat_number_match", "ext_place_id_match", "ext_levenstein_name_address", "ext_match_details", "int_levenstein_name_address","int_vat_number_match","int_customer_number_match","match_details",
    "customer_number_Occurrence")
    .orderBy(F.desc(F.col("count_dim_sellout_cust_id")), F.col("customer_name"))
    .filter(F.col("customer_number_Occurrence") >= 2)
    # .filter(F.col("customer_number").isin("10052"))
    # .filter(F.col("ext_BPR_match").isNotNull())
    .display()
)

In [0]:
df_matched_ext.select("dim_sellout_cust_id").display()

In [0]:
from pyspark.sql import functions as F

distinct_ids = (
    df_matched_ext
    .groupBy("dim_sellout_cust_id")
    .agg(
        F.collect_set("C4S_account_id").alias("distinct_C4S_ids"),
        F.collect_set("direct_c4S_account_id").alias("distinct_direct_C4S_ids")
    )
    # Union both arrays and remove duplicates
    .withColumn(
        "union_ids",
        F.array_distinct(F.flatten(F.array("distinct_C4S_ids", "distinct_direct_C4S_ids")))
    )
    # Count the size of the union
    .withColumn("union_count", F.size(F.col("union_ids")))
)

display(distinct_ids)

In [0]:
df_joined = (
    df_matched_ext.alias("l")
    .join(
        df_c4s.alias("r"),
        on = F.expr("l.C4S_account_id == r.Account_Id"),
        how = "inner"
    )
    .selectExpr(
        "l.*",
        "r.Name",
        "r.Street",
        "r.Postal_Code"
    )
)

In [0]:
(
    df_joined
    .select("customer_name","Name","customer_address","Street", "vat_number", "direct_c4S_account_id", "C4S_account_id")
    .display()
)